In [1]:
import numpy as np
from sklearn import datasets

import torch
import torch.nn as nn
import torch.optim as optim

import torchbnn as bnn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
X = pd.read_csv('data/supervised_x_transform_onehot.csv')
Y = pd.read_csv('data/supervised_y_transform.csv')

In [3]:
x, y = torch.from_numpy(np.array(X)).float(), torch.from_numpy(np.array(Y).squeeze()).long()
print(x.shape, y.shape)

torch.Size([1933, 5190]) torch.Size([1933])


In [43]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=5190, out_features=100000),
    #nn.ReLU(),
    nn.Sigmoid(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=100000, out_features=4),
)

ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [ ]:
kl_weight = 0.1

for step in range(10):
    pre = model(x)
    ce = ce_loss(pre, y)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
_, predicted = torch.max(pre.data, 1)
total = y.size(0)
correct = (predicted == y).sum()
print('- Accuracy: %f %%' % (100 * float(correct) / total))
print('- CE : %2.2f, KL : %2.2f' % (ce.item(), kl.item()))

In [41]:
np.unique(Y)

array([0, 1, 2, 3], dtype=int64)

In [42]:
predicted

tensor([2, 2, 2,  ..., 2, 2, 2])

In [25]:
predicted

tensor([2, 2, 2,  ..., 2, 2, 2])